In [ ]:
from train import train_and_validate_hierarchical

train_csv = "/mnt/Data/hackathon/final_train.csv"
val_csv = "/mnt/Data/hackathon/final_valid.csv"
train_and_validate_hierarchical(train_csv, val_csv, epochs=100, batch_size=8, train_sweeps=1, val_sweeps=8, save_path='checkpoints/best_model.pth')

In [ ]:
from infer import infer_test

# Call it directly
infer_test(
    test_csv='/mnt/Data/hackathon/final_test.csv',
    model_path='checkpoints/best_model.pth',
    n_sweeps_test=8,
    output_csv='outputs/test_predictions.csv'
)


In [ ]:
import argparse
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")

from evaluate_metrics import compute_metrics

compute_metrics(
    pred_csv="outputs/test_predictions.csv",
    gt_csv="/mnt/Data/hackathon/final_valid.csv"
)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")

def compute_metrics_by_trimester(pred_csv, gt_csv):
    # Load data
    pred_df = pd.read_csv(pred_csv)
    gt_df = pd.read_csv(gt_csv)

    # Ensure columns exist
    if 'study_id' not in pred_df.columns or 'ga' not in pred_df.columns:
        raise ValueError("Prediction CSV must contain 'study_id' and 'ga' columns.")
    if 'study_id' not in gt_df.columns or 'ga' not in gt_df.columns:
        raise ValueError("Ground truth CSV must contain 'study_id' and 'ga' columns.")

    # Merge on study_id
    merged = pd.merge(gt_df, pred_df, on='study_id', suffixes=('_true', '_pred'))
    if merged.empty:
        raise ValueError("No matching study_id values found between CSVs.")

    y_true = merged['ga_true']
    y_pred = merged['ga_pred']

    # Determine trimester thresholds (based on ground truth)
    max_ga = y_true.max()
    min_ga = y_true.min()
    step = (max_ga - min_ga) / 3

    trimester_bins = [min_ga, min_ga + step, min_ga + 2*step, max_ga]
    trimester_labels = ['Trimester 1', 'Trimester 2', 'Trimester 3']

    merged['trimester'] = pd.cut(y_true, bins=trimester_bins, labels=trimester_labels, include_lowest=True)

    # Store results
    results = []

    for trimester in trimester_labels:
        df_t = merged[merged['trimester'] == trimester]
        if df_t.empty:
            continue

        y_t_true = df_t['ga_true'].values
        y_t_pred = df_t['ga_pred'].values

        mae = mean_absolute_error(y_t_true, y_t_pred)
        mse = mean_squared_error(y_t_true, y_t_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_t_true, y_t_pred)
        corr, _ = pearsonr(y_t_true, y_t_pred)

        results.append({
            "Trimester": trimester,
            "Samples": len(y_t_true),
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R²": r2,
            "Corr": corr
        })

    results_df = pd.DataFrame(results)
    print("\n📊 Metrics by Trimester")
    print(results_df.round(4).to_string(index=False))

    # Overall metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    corr, _ = pearsonr(y_true, y_pred)

    print("\n📈 Overall Metrics")
    print("---------------------------")
    print(f"MAE   : {mae:.4f}")
    print(f"MSE   : {mse:.4f}")
    print(f"RMSE  : {rmse:.4f}")
    print(f"R²    : {r2:.4f}")
    print(f"Corr  : {corr:.4f}")
    print("---------------------------")
    print(f"Samples evaluated: {len(y_true)}")

    return results_df


In [ ]:
trimester_results = compute_metrics_by_trimester(
    pred_csv="outputs/test_predictions.csv",
    gt_csv="/mnt/Data/hackathon/final_valid.csv"
)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")

def compute_metrics_by_site(pred_csv, gt_csv):
    # Load data
    pred_df = pd.read_csv(pred_csv)
    gt_df = pd.read_csv(gt_csv)

    # Ensure columns exist
    if 'study_id' not in pred_df.columns or 'ga' not in pred_df.columns:
        raise ValueError("Prediction CSV must contain 'study_id' and 'ga' columns.")
    if 'study_id' not in gt_df.columns or 'ga' not in gt_df.columns:
        raise ValueError("Ground truth CSV must contain 'study_id' and 'ga' columns.")
    if 'site' not in gt_df.columns:
        raise ValueError("Ground truth CSV must contain a 'site' column for site-based evaluation.")

    # Merge predictions with ground truth
    merged = pd.merge(gt_df, pred_df, on='study_id', suffixes=('_true', '_pred'))
    if merged.empty:
        raise ValueError("No matching study_id values found between CSVs.")

    # Extract arrays
    y_true = merged['ga_true']
    y_pred = merged['ga_pred']

    # Compute metrics per site
    site_results = []
    for site, df_site in merged.groupby('site'):
        y_s_true = df_site['ga_true'].values
        y_s_pred = df_site['ga_pred'].values

        mae = mean_absolute_error(y_s_true, y_s_pred)
        mse = mean_squared_error(y_s_true, y_s_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_s_true, y_s_pred)
        corr, _ = pearsonr(y_s_true, y_s_pred)

        site_results.append({
            "Site": site,
            "Samples": len(y_s_true),
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R²": r2,
            "Corr": corr
        })

    results_df = pd.DataFrame(site_results)

    print("\n📊 Metrics by Site")
    print(results_df.round(4).to_string(index=False))

    # Compute overall metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    corr, _ = pearsonr(y_true, y_pred)

    print("\n📈 Overall Metrics")
    print("---------------------------")
    print(f"MAE   : {mae:.4f}")
    print(f"MSE   : {mse:.4f}")
    print(f"RMSE  : {rmse:.4f}")
    print(f"R²    : {r2:.4f}")
    print(f"Corr  : {corr:.4f}")
    print("---------------------------")
    print(f"Samples evaluated: {len(y_true)}")

    return results_df


In [ ]:
site_results = compute_metrics_by_site(
    pred_csv="outputs/test_predictions.csv",
    gt_csv="/mnt/Data/hackathon/final_valid.csv"
)


In [ ]:
%%shell
tensorboard --logdir=logs --port=6006

ssh -L 6006:localhost:6006 team-ua@113.199.208.74

In [ ]:
from infer import predict_ga

model_path = "checkpoints/best_model.pth"
test_csv = "path/to/test.csv"
predictions = predict_ga(model_path, test_csv)

In [ ]:
from train import train_model

best_mae, best_loss = train_model(
    model_type='resnet18',
    train_csv="/mnt/Data/hackathon/final_train.csv",
    val_csv="/mnt/Data/hackathon/final_valid.csv",
    epochs=50,
    batch_size=8,
    reduced_dim=128,
    fine_tune_backbone=True
)

In [ ]:
from train import train_model

best_mae, best_loss = train_model(
    model_type='convnext_tiny',
    train_csv="/mnt/Data/hackathon/final_train.csv",
    val_csv="/mnt/Data/hackathon/final_valid.csv",
    epochs=50,
    batch_size=8,
    learning_rate=3e-5,
    reduced_dim=384,
    fine_tune_backbone=True
)

In [ ]:
# You can also call it programmatically
from train import train_model

# Train ResNet50
best_mae, best_loss = train_model(
    model_type='convnext_tiny',
    train_csv="/mnt/Data/hackathon/final_train.csv",
    val_csv="/mnt/Data/hackathon/final_valid.csv",
    epochs=50,
    batch_size=8,
    reduced_dim=256,
    fine_tune_backbone=True
)

In [ ]:
# train_sequential.py
import time
from train import train_model

# List of models to train in hierarchical mode
models_to_train = [
    {
        'model_type': 'resnet18',
        'architecture': 'hierarchical',
        'train_sweeps': 2,
        'batch_size': 8,
        'learning_rate': 1e-4,
        'reduced_dim': 128,
        'num_heads': 4,
        'save_dir': 'checkpoints_hierarchical',
        'experiment_name': 'hierarchical_resnet18'
    },
    {
        'model_type': 'resnet50',
        'architecture': 'hierarchical',
        'train_sweeps': 2,
        'batch_size': 4,
        'learning_rate': 1e-4,
        'reduced_dim': 256,
        'num_heads': 4,
        'save_dir': 'checkpoints_hierarchical',
        'experiment_name': 'hierarchical_resnet50'
    },
    {
        'model_type': 'convnext_tiny',
        'architecture': 'hierarchical',
        'train_sweeps': 2,
        'batch_size': 8,
        'learning_rate': 3e-5,
        'reduced_dim': 384,
        'num_heads': 4,
        'save_dir': 'checkpoints_hierarchical',
        'experiment_name': 'hierarchical_convnext'
    }
]

print("Starting sequential hierarchical model training...")
print("="*60)

for i, config in enumerate(models_to_train):
    print(f"\nTraining Model {i+1}/{len(models_to_train)}: {config['model_type']}")
    print(f"   Architecture: {config['architecture']}")
    print(f"   Batch size: {config['batch_size']}")
    print(f"   Learning rate: {config['learning_rate']}")
    print(f"   Reduced dim: {config['reduced_dim']}")
    print(f"   Train sweeps: {config['train_sweeps']}")
    print("-" * 40)
    
    try:
        best_mae, best_loss = train_model(
            model_type=config['model_type'],
            architecture=config['architecture'],
            train_csv="/mnt/Data/hackathon/final_train.csv",
            val_csv="/mnt/Data/hackathon/final_valid.csv",
            epochs=50,
            batch_size=config['batch_size'],
            train_sweeps=config['train_sweeps'],
            n_sweeps_val=8,
            learning_rate=config['learning_rate'],
            reduced_dim=config['reduced_dim'],
            num_heads=config['num_heads'],
            fine_tune_backbone=True,
            pretrained=True,
            save_dir=config['save_dir']
        )
        
        print(f"{config['model_type']} completed!")
        print(f"   Best MAE: {best_mae:.4f}")
        print(f"   Best Loss: {best_loss:.4f}")
        
    except Exception as e:
        print(f"{config['model_type']} failed: {e}")
    
    # Wait between models (optional)
    if i < len(models_to_train) - 1:
        print("\nWaiting 5 seconds before next model...")
        time.sleep(5)

print("\n" + "="*60)
print("All hierarchical models training completed!")
print("="*60)

Starting sequential hierarchical model training...

Training Model 1/3: resnet18
   Architecture: hierarchical
   Batch size: 8
   Learning rate: 0.0001
   Reduced dim: 128
   Train sweeps: 2
----------------------------------------
Using device: cuda
Training hierarchical_resnet18 model...
Training with 2 sweep(s) per study
Loading datasets...
Train samples: 395
Val samples: 50
Train batches: 50
Val batches: 7
Creating hierarchical_resnet18 model...
Total parameters: 12,360,385
Trainable parameters: 12,360,385

Starting training...



Epoch 1/50:
  Train Loss: 150.0873 | Train MAE: 150.0873
  Val Loss:   163.1494 | Val MAE:   163.1494
  Learning Rate: 1.00e-04
✅ Saved new best model (Val MAE: 163.1494) to checkpoints_hierarchical/best_model_hierarchical_resnet18.pth
